In [10]:
from hashlib import sha256
import json
import time
from datetime import date, datetime


In [26]:
class Block:
    def __init__(self, ind, details, timestamp, prev_hash, nonce=0):
        self.ind = ind
        self.details = details
        self.creation_time = timestamp
        self.prev_hash = prev_hash
        self.seq_number = nonce

    def string_to_hash(self):
        data_as_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(data_as_string.encode()).hexdigest()

In [121]:
class Blockchain: 
    def __init__(self):
        self.blockchain = []
        self.initialize_blockchain()

    def initialize_blockchain(self):
        genesis_block = Block(0, [{'Forest_1':{'Phase':'Forest audit',
            'auditor id':'123',
            'audit date': str(date.today().strftime("%B %d, %Y")),
            'geo_location': {'country_code': 'FI','latitude': 63.11569,'longitude': 21.7303},
            'site_audits': '2',
            'certificates': ['PEFC','FSC']}}], str(datetime.now()), "0")
        genesis_block.hash = genesis_block.string_to_hash()
        self.blockchain.append(genesis_block)

    @property
    def chain_end(self):
        return self.blockchain[-1]
        
    difficulty = 3
    proof_string = "".join([str(i) for i in range(difficulty)])

    def proof_of_work(self, block):
        computed_hash = block.string_to_hash()
        while not computed_hash.startswith(Blockchain.proof_string):
            block.seq_number += 1
            computed_hash = block.string_to_hash()
        return computed_hash

    def add_block(self, block, proof):
        previous_hash = self.chain_end.hash
        if previous_hash != block.prev_hash:
            return False
        if not self.validate_proof(block, proof):
            return False
        block.hash = proof
        self.blockchain.append(block)
        return True

    def validate_proof(self, block, block_hash):
        return (block_hash.startswith(Blockchain.proof_string) and
              block_hash == block.string_to_hash())

    def add_details(self, detail):
        chain_end = self.chain_end
        new_block = Block(ind=chain_end.ind + 1,
                        details = detail,
                        timestamp=str(datetime.now()),
                        prev_hash=chain_end.hash)
        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)
        return new_block.creation_time
    
    def search_company_details(self, name):
        for i in range(len(self.blockchain)):
            if i == 0:
                continue
            try:
                print(self.blockchain[i].details[name])
            except KeyError:
                continue
                
    def get_chain(self):
        print('Blockchain:')
        for i in self.blockchain:
            print('Index: ' + str(i.ind))
            print('\tDetails: ' + str(i.details))
            print('\tTimestamp: ' + str(i.creation_time))
            print('\tHash: ' + str(i.hash))


In [116]:
Forest_1 = {'Forest_1':{'Phase':'Forest audit',
            'auditor id':'123',
            'audit date': str(date.today().strftime("%B %d, %Y")),
            'geo_location': {'country_code': 'FI','latitude': 63.11569,'longitude': 21.7303},
            'site_audits': '2',
            'certificates': ['PEFC','FSC']}}
Forest_2 = {'Forest_2':{'Phase':'Forest audit',
            'auditor id':'456',
            'audit date': str(date.today().strftime("%B %d, %Y")),
            'geo_location': {'country_code': 'FI','latitude': 63.11569,'longitude': 21.7303},
            'site_audits': '3',
            'certificates': ['PEFC','FSC']}}

Harvesting_contractor_1 = {'Harvesting_contractor_1':{'Phase':'Harvesting',
                           'business_id':'123',
                           'harvesting date': str(date.today().strftime("%B %d, %Y")),
                           'geo_location': {'country_code': 'FI','latitude': 63.11569,'longitude': 21.7303},
                           'certificates': ['ABC','DEF'],
                           'quantity': 100,
                           'logistics_supplier_id':'123'}}
Harvesting_contractor_2 = {'Harvesting_contractor_2':{'Phase':'Harvesting',
                           'business_id':'456',
                           'harvesting date': str(date.today().strftime("%B %d, %Y")),
                           'geo_location': {'country_code': 'FI','latitude': 63.11569,'longitude': 21.7303},
                           'certificates': ['ABC','DEF'],
                           'quantity': 200,
                           'logistics_supplier_id':'456'}}

Supplier_1 = {'Spplier_1':{'Phase':'Supply to manuf.',
              'business_id':'789',
              'entry date': str(date.today().strftime("%B %d, %Y")),
              'entry':'verified',
              'delivery date':str(date.today().strftime("%B %d, %Y")),
              'exit':'verified',
              'vehicle load utilization_%':75,
              'sub-supplier_ids':['123','456'],
              'quantity': 100}}
Supplier_2 = {'Supplier_2':{'Phase':'Supply to manuf.',
              'business_id':'123',
              'entry date': str(date.today().strftime("%B %d, %Y")),
              'entry':'verified',
              'delivery date':str(date.today().strftime("%B %d, %Y")),
              'exit':'verified',
              'vehicle load utilization_%':60,
              'sub-suppliers':['123','456'],
              'quantity': 200}}

Manufacturer_1 = {'Manufacturer_1':{'Phase':'Manuf.',
                  'business_id':'456',
                  'entry date':str(date.today().strftime("%B %d, %Y")),
                  'entry':'verified',
                  'manuf. date': str(date.today().strftime("%B %d, %Y")),
                  'delivery date':str(date.today().strftime("%B %d, %Y")),
                  'exit':'verified',
                  'Management systems' : ['ISO 14001', 'ISO 9001'],
                  'payment':'approved',
                  'received quantity': 100,
                  'distributed quantity': 500}}
Manufacturer_2 = {'Manufacturer_2':{'Phase':'Manuf.',
                  'business_id':'789',
                  'entry date':str(date.today().strftime("%B %d, %Y")),
                  'entry':'verified',
                  'manuf. date': str(date.today().strftime("%B %d, %Y")),
                  'delivery date':str(date.today().strftime("%B %d, %Y")),
                  'exit':'verified',
                  'Management systems' : ['ISO 14001', 'ISO 9001'],
                  'payment':'approved',
                  'received quantity': 200,
                  'distributed quantity': 1000}}

Distributor_1 = {'Distributor_1':{'Phase':'Distr. from manuf.',
                 'business_id':'123',
                 'entry date':str(date.today().strftime("%B %d, %Y")),
                 'entry':'verified',
                 'distr. date': str(date.today().strftime("%B %d, %Y")),
                 'exit':'verified',
                 'vehicle load utilization_%':65,
                 'delivery date':'',
                 'payment':'approved',
                 'quantity':500}}
Distributor_2 = {'Distributor_2':{'Phase':'Distr. from manuf.',
                 'business_id':'456',
                 'entry date':str(date.today().strftime("%B %d, %Y")),
                 'entry':'verified',
                 'distr. date': str(date.today().strftime("%B %d, %Y")),
                 'exit':'verified',
                 'vehicle load utilization_%':70,
                 'delivery date':'',
                 'payment':'approved',
                 'quantity':1000}}

Customer_1 = {'Customer_1':{'Phase':'Customer',
              'business_id':'789',
              'entry time':str(date.today().strftime("%B %d, %Y")),
              'entry':'verified',
              'payment':'approved',
              'quantity':500}}
Customer_2 = {'Customer_2':{'Phase':'Customer',
              'business_id':'123',
              'entry time':str(date.today().strftime("%B %d, %Y")),
              'entry':'verified',
              'payment':'approved',
              'quantity':1000}}

In [122]:
chain = Blockchain()

In [123]:
chain.add_details(Forest_1)
chain.add_details(Harvesting_contractor_1)
chain.add_details(Supplier_1)
chain.add_details(Manufacturer_1)
chain.add_details(Distributor_1)
chain.add_details(Customer_1)

'2022-03-30 12:32:33.744875'

In [124]:
chain.get_chain()

Blockchain:
Index: 0
	Details: []
	Timestamp: 2022-03-30 12:32:33.395435
	Hash: 355923cedfecbfa21be03caa66666b2d9f58ee161067a1cc8865b39aee6b3ecc
Index: 1
	Details: {'Forest_1': {'Phase': 'Forest audit', 'auditor id': '123', 'audit date': 'March 30, 2022', 'geo_location': {'country_code': 'FI', 'latitude': 63.11569, 'longitude': 21.7303}, 'site_audits': '2', 'certificates': ['PEFC', 'FSC']}}
	Timestamp: 2022-03-30 12:32:33.571755
	Hash: 0120f95d47156389f1230a33d1a31229de4ec92e02073106d7d46cb3e3615998
Index: 2
	Details: {'Harvesting_contractor_1': {'Phase': 'Harvesting', 'business_id': '123', 'harvesting date': 'March 30, 2022', 'geo_location': {'country_code': 'FI', 'latitude': 63.11569, 'longitude': 21.7303}, 'certificates': ['ABC', 'DEF'], 'quantity': 100, 'logistics_supplier_id': '123'}}
	Timestamp: 2022-03-30 12:32:33.664569
	Hash: 012c39ba4b4bb1d5877bb119f403cadddfc804905c4c2267cf3328829e042b26
Index: 3
	Details: {'Spplier_1': {'Phase': 'Supply to manuf.', 'business_id': '789', 'en

In [125]:
name='Distributor_1'
chain.search_company_details(name)

{'Phase': 'Distr. from manuf.', 'business_id': '123', 'entry date': 'March 30, 2022', 'entry': 'verified', 'distr. date': 'March 30, 2022', 'exit': 'verified', 'vehicle load utilization_%': 65, 'delivery date': '', 'payment': 'approved', 'quantity': 500}
